In [1]:
#Import the librarries
import pandas as pd
import requests
!conda install -c conda-forge folium
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.4.0-py_0        conda-forge
    folium:          

Here we are interested in the High Schools. Hence we pass the category id of the High school as a parameter.
Category Id of the high school can be found at the FourSquare API documentation.

In [2]:
#Construct the Foursquare credentials and parameters

CLIENT_ID = '24I4TTWLBR5VXYNKGFMSNELLRYMNCCHF10HUUOZ2LE2ZJPMU' 
CLIENT_SECRET = 'QCANHSJ5WHTTPKK5NDCCA2WJLFDLGW2QTRNAVNXLJZ3TLTJO' 
VERSION = '20180604'
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
   "v": VERSION,
    "categoryId": "4bf58dd8d48988d13d941735", # category id of high school
    "near": "san diego, california",
    "radius": 10000,
    "limit": 50}


In [3]:
#get the results
data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)
d = data.json()["response"]

In [4]:
#Get the items from the response and print a sample json result
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 42


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bbe5d61b083a5933ee5a1e9',
  'name': 'San Diego SCPA',
  'location': {'address': '2425 Dusk Dr',
   'crossStreet': 'btw Alta View Dr & Parkside Ave',
   'lat': 32.67987864048685,
   'lng': -117.04860949087335,
   'labeledLatLngs': [{'label': 'display',
     'lat': 32.67987864048685,
     'lng': -117.04860949087335}],
   'postalCode': '92139',
   'cc': 'US',
   'city': 'San Diego',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['2425 Dusk Dr (btw Alta View Dr & Parkside Ave)',
    'San Diego, CA 92139',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d13d941735',
    'name': 'High School',
    'pluralName': 'High Schools',
    'shortName': 'High School',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
     'suffix': '.png'},
    'primary': True}],
  'phot

In [5]:
#Construct the dataframe from the json response
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    if not "address" in location:
        continue
    address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
    
df = pd.DataFrame(df_raw, columns=["VenueID","SchoolName", "ShortName", "Address", "PostalCode", "Latitude", "Longitude"])
print("Found %i schools" % len(df))
df.head()

Found 38 schools


,VenueID,SchoolName,ShortName,Address,PostalCode,Latitude,Longitude
0,4bbe5d61b083a5933ee5a1e9,San Diego SCPA,High School,2425 Dusk Dr,92139,32.679879,-117.048609
1,4b548e8ff964a520e5bf27e3,Helix Charter High School,High School,7323 University Ave,91942,32.754317,-117.037150
2,4bba201d7421a593db6bc340,Garfield High School,High School,1255 16th St,92101,32.718490,-117.149067
3,4dd293b61838a751965e6ca4,SDEMC,High School,1425 Russ Blvd Ste T112D,92101,32.719405,-117.151360
4,4bc8950d6501c9b6aa664029,San Diego High School,High School,1405 Park Blvd,92101,32.721871,-117.152180


Create map of San Diego using latitude and longitude values
Label the points using the postal code so that it will be easier for us to locate the venues.

In [6]:
sd_center = d["geocode"]["center"]

from folium import plugins

map_sd = folium.Map(location=[sd_center["lat"], sd_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["PostalCode"], parse_html=True)
        folium.CircleMarker(
            [row["Latitude"], row["Longitude"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_sd)

add_markers(df)
hm_data = df[["Latitude", "Longitude"]].values.tolist()
map_sd.add_child(plugins.HeatMap(hm_data))

map_sd

From the map we can see that the max schools are around the area with postal code 92106
The following is the filtered dataset with the postal code

In [7]:
df[df['PostalCode'].str.contains("92106")]

,VenueID,SchoolName,ShortName,Address,PostalCode,Latitude,Longitude
11,59ca7a7b0f013c03fa8ce05c,High Tech High Forum,High School,2150 Cushing Rd,92106,32.732016,-117.217650
12,4c2102544d4a20a142a89b70,High Tech High,High School,2861 Womble Rd,92106,32.734872,-117.217801
14,4fb26f9be4b02fcbfcc5fe31,High Tech High International,High School,2855 Farragut Rd,92106,32.733300,-117.219016
15,4cf57812c020b60cd2d61a75,The Rock Academy,High School,2277 Rosecrans St.,92106,32.736726,-117.217910
16,529f6bde498e0fde6ec47182,High Tech High Media Arts,High School,2230 Truxtun Rd,92106,32.734595,-117.219647
19,4be3c20663609c747ff31bff,Point Loma High School,High School,2335 Chatsworth Blvd,92106,32.739340,-117.226234


We can pick a random latitude and the logitude information, to propose the preferable location of residence.

In [8]:
lat = 32.736726
lng = -117.217910
map_residence = folium.Map(location=[lat, lng], zoom_start=14)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Preferred location of residence",
    color='red',
    fill=True,
    fill_color='#3186ee',
    fill_opacity=0.7,
    parse_html=False).add_to(map_residence)
map_residence

Another way to get a precise information on the schools is to use the ratings for each venue. 
Since we are using Regular API we couldn't fetch that information.

So we stop our exercise here